In [1]:
import numpy as np
import pandas as pd
import os
import nltk
import pickle
import re

from PIL import Image

from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer, util

In [2]:
# get songs data from test set

with open('../App_stcloud/pickle_objects/embeddings_df.pickle', 'rb') as f:
        l_train_test = pickle.load(f)

songs_set = l_train_test

In [3]:
# uncomment to extract .csv data
# get the cleaned songs data
# songs_set = pd.read_csv('../songsdata/cleaned_data/cleaned_songs.csv')

In [4]:
songs_set['lyrics'].head()

51788    Break Apart Her Heart Lyrics[Verse 1]\nWhen yo...
31697    Alphabet Rap Lyrics(A, B, C, D\nA\nA)\n\nA, B,...
49885    Can It Be All So Simple (Remix) Lyrics[Skit: R...
3349     Chance Lyrics[Verse 1]\nAll the rain came down...
53551    Paper Face LyricsI still have the photograph t...
Name: lyrics, dtype: object

In [5]:
# get songs lyrics set
lyrics_set = songs_set['lyrics']

In [11]:
# pickle the sets
l_pickle = [songs_set, lyrics_set]
with open('../App_stcloud/pickle_objects/song_lyrics_set_test.pickle', 'wb') as f:
    pickle.dump(l_pickle, f)

In [6]:
# get embedding function
def embeddings_gen(lyrics_set, model_name = "all-distilroberta-v1"):
    # Create mdoel
    model = SentenceTransformer(model_name)
    # Create bag of lyrics lines with their corresponding song_ids
    l_lyrics_lines =[]
    l_song_idx =[]
    for idx in lyrics_set.index:
        lyrics = lyrics_set[idx]
        lyrics_lines = re.split('\n',lyrics)
        # Condition: do not include lyrics lines that are more than 512 tokens
        if any(len(word_tokenize(x)) >= 512 for x in lyrics_lines):
            continue
        l_lyrics_lines.extend(lyrics_lines)
        l_song_idx.extend([idx] * len(lyrics_lines))

    # For invert indexing //Store related song ids as np array
    arr_song_idx = np.array(l_song_idx)
    # Store lyrics lines as np array
    arr_lyrics_idx = np.array(l_lyrics_lines)

    embeddings = model.encode(l_lyrics_lines, convert_to_tensor=True)

    return embeddings, arr_song_idx, arr_lyrics_idx


In [7]:
# uncomment if further down-sampling on test set is needed
#with open('../pickle_objects/song_lyrics_set.pickle', 'rb') as f:
    # l_pickle = pickle.load(f)

#downsample_size = 

#lyrics_set = l_pickle[1].sample(downsample_size, random_state=42)

In [14]:
# # transform lyrics to vector embedding
# # get the vector embedding of the sampling 1000 dataset from the 100k songs data >> preprocess_lyrics.ipynb
# with open('../App/pickle_objects/embeddings_indices.obj', 'rb') as f:
#     embeddings, arr_song_idx, arr_lyrics_idx = pickle.load(f)

# with open('../App/pickle_objects/sample_song_lyrics_set.obj', 'rb') as f:
#     songs_set, lyrics_set = pickle.load(f)

# # getting songs index to be trained with bertopic
# songs_idx = sorted(list(set(arr_song_idx)))

# # getting songs in the index
# sample_songs_set = songs_set.loc[songs_set.index.isin(songs_idx)]
# lyrics_set = sample_songs_set['lyrics']

In [10]:
# gen embedding for based model
# embeddings, arr_song_idx, arr_lyrics_idx = embeddings_gen(lyrics_set)

In [8]:
# gen embedding from fine-tuned model
embeddings, arr_song_idx, arr_lyrics_idx = embeddings_gen(lyrics_set, model_name = '../App_stcloud/pickle_objects/ft_model_full_annotations')

In [9]:
with open('../App_stcloud/pickle_objects/embeddings_test.pickle', 'wb') as f:
    pickle.dump(embeddings, f)

In [10]:
with open('../App_stcloud/pickle_objects/arr_song_idx_test.pickle', 'wb') as f:
    pickle.dump(arr_song_idx, f)

In [11]:
with open('../App_stcloud/pickle_objects/arr_lyrics_idx_test.pickle', 'wb') as f:
    pickle.dump(arr_lyrics_idx, f)

In [23]:
# get embedding function
def embeddings_gen_clip(lyrics_set, model_name = "clip-ViT-B-32"):
    # Create mdoel
    model = SentenceTransformer(model_name)
    # Create bag of lyrics lines with their corresponding song_ids
    l_lyrics_lines =[]
    l_song_idx =[]
    for idx in lyrics_set.index:
        lyrics = lyrics_set[idx]
        lyrics_lines = re.split('\n',lyrics)
        # Condition: CLIP max token limit: 77; Do not include lyrics lines that are more than 77 tokens
        if any(len(word_tokenize(x)) >= 77 for x in lyrics_lines):
            continue
        l_lyrics_lines.extend(lyrics_lines)
        l_song_idx.extend([idx] * len(lyrics_lines))

    # For invert indexing //Store related song ids as np array
    arr_song_idx = np.array(l_song_idx)
    # Store lyrics lines as np array
    arr_lyrics_idx = np.array(l_lyrics_lines)

    embeddings = model.encode(l_lyrics_lines, convert_to_numpy=True)

    return embeddings, arr_song_idx, arr_lyrics_idx

embeddings_clip, arr_song_idx_clip, arr_lyrics_idx_clip = embeddings_gen_clip(lyrics_set)

ftfy or spacy is not installed using BERT BasicTokenizer instead of ftfy.


In [ ]:
with open('../App_stcloud/pickle_objects/embeddings_clip.pickle', 'wb') as f:
    pickle.dump(embeddings_clip, f)

In [ ]:
with open('../App_stcloud/pickle_objects/arr_song_idx_clip.pickle', 'wb') as f:
    pickle.dump(arr_song_idx_clip, f)

In [ ]:
with open('../App_stcloud/pickle_objects/arr_lyrics_idx_clip.pickle', 'wb') as f:
    pickle.dump(arr_lyrics_idx_clip, f)

In [7]:
model = SentenceTransformer('clip-ViT-B-32')

KeyError: 'model_name'